In [1]:
import tweepy
import pandas as pd
import pymongo
import datetime
from pymongo import MongoClient, ASCENDING, TEXT

In [2]:
consumer_key = 'jCVZcxudnaBiO8IjDIKburzU8'
consumer_secret = 'eTKxXOT1X3kGuFzpSIFewDfO0KASKcMSqfTLmiZqwFoDc8wjtH'
access_token = '1294569170135212032-yFNqooPaM1xWRwMS1dOzULaiSap2sv'
access_secret = 'Z3TFX8uNL6ulXJDLBNzPYRYCgJcikZNnheSfWSMIhgJaT'
bearer_token ='AAAAAAAAAAAAAAAAAAAAAJdtlAEAAAAA4u3gYVNv8sBwdYczEOY%2FW5hcDXo%3Duu3uKcVwFQ6bnYkTxPHIiQSkKT9fDWafMSe338rbSJzd6X305K'

In [3]:
client = MongoClient('mongodb://localhost:27017')
db = client.twitter
tweet = db.tweet
twitter_collection = db.twitter_collection
tweet_1 = db.tweet_1

In [4]:
tweet.index_information()

{}

In [5]:
twitter_collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]}}

In [6]:
# authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

In [7]:
print(twitter_collection.estimated_document_count())

259


In [8]:
search_results = api.search_tweets( count= 15,q= 'College', lang = 'en', result_type = 'popular', until = '2023-01-09')
t_count=0
for i in search_results:
    try : 
        twitter_collection.insert_one(i._json) 
        t_count+=1
        print("Tweet Saved in Collecction, Tweet_count: ",t_count)
    except : 
        print("Not Saved in collection, Tweet Id is not Unique")
        pass

Tweet Saved in Collecction, Tweet_count:  1
Tweet Saved in Collecction, Tweet_count:  2
Tweet Saved in Collecction, Tweet_count:  3
Tweet Saved in Collecction, Tweet_count:  4
Tweet Saved in Collecction, Tweet_count:  5
Tweet Saved in Collecction, Tweet_count:  6
Tweet Saved in Collecction, Tweet_count:  7
Tweet Saved in Collecction, Tweet_count:  8
Tweet Saved in Collecction, Tweet_count:  9
Tweet Saved in Collecction, Tweet_count:  10
Tweet Saved in Collecction, Tweet_count:  11
Tweet Saved in Collecction, Tweet_count:  12
Tweet Saved in Collecction, Tweet_count:  13
Tweet Saved in Collecction, Tweet_count:  14
Tweet Saved in Collecction, Tweet_count:  15


In [9]:
print("Number of Documents in Collection: ",twitter_collection.estimated_document_count())

Number of Documents in Collection:  274


In [10]:
for i in twitter_collection.find({"text":{"$regex":"College"}}).limit(10):
    try:
        print("_ _ _ _ _ _ _ _ _ _ _ ")
        print("User name:",i["user"]["name"])
        print("Text:",i["text"])
    except:
        pass

_ _ _ _ _ _ _ _ _ _ _ 
User name: Rep. Anna V. Eskamani 🔨
Text: So much attention on what's happening at @NewCollegeofFL but not enough attention of how this sits in the larger fr… https://t.co/ZIZfrhTbJI
_ _ _ _ _ _ _ _ _ _ _ 
User name: Ashok Gehlot
Text: Talking to media at SMS Medical College | January 08. https://t.co/sjiIq7dkQu
_ _ _ _ _ _ _ _ _ _ _ 
User name: Dinesh D'Souza
Text: The Ontario College of Psychologists has ordered Jordan Peterson to mandatory social media communication retraining… https://t.co/tniUeEcVXo
_ _ _ _ _ _ _ _ _ _ _ 
User name: Rebel News
Text: Save Dr. Jordan Peterson!

The College of Psychologists of Ontario is trying to cancel Dr. Jordan Peterson — help f… https://t.co/zjS0hbgKtU
_ _ _ _ _ _ _ _ _ _ _ 
User name: Rep. Anna V. Eskamani 🔨
Text: So much attention on what's happening at @NewCollegeofFL but not enough attention of how this sits in the larger fr… https://t.co/ZIZfrhTbJI
_ _ _ _ _ _ _ _ _ _ _ 
User name: Ashok Gehlot
Text: Talking to media a

In [11]:
print(i.keys())

dict_keys(['_id', 'created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang'])


In [12]:
#now we need to do sentimental of our data using pymongo

#we need to create a new collection for this

for i in twitter_collection.find({"text":{"$regex":"College"}}).limit(10):
    try:
        tweet_1.insert_one(i)
    except:
        pass

In [13]:
tweet_1.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'text_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('text', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [14]:

tweet_1.create_index([('text', TEXT)], default_language='english')

'text_text'

In [15]:
tweet_1.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'text_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('text', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [16]:
tweet_1.estimated_document_count()

15

In [17]:
tweet_1.find_one()

{'_id': ObjectId('63bc32125938cef927729a1e'),
 'created_at': 'Sun Jan 08 10:58:19 +0000 2023',
 'id': 1612040998988746754,
 'id_str': '1612040998988746754',
 'text': "Letters: Sorry Erin O'Toole, but those 'f--k Trudeau' flags are giving voice to democracy https://t.co/s2P8rdsk4Y",
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/s2P8rdsk4Y',
    'expanded_url': 'https://nationalpost.com/opinion/those-f-k-trudeau-flags-are-giving-voice-to-democracy?utm_term=Autofeed&utm_medium=Social&utm_source=Twitter#Echobox=1673175273',
    'display_url': 'nationalpost.com/opinion/those-…',
    'indices': [90, 113]}]},
 'metadata': {'result_type': 'popular', 'iso_language_code': 'en'},
 'source': '<a href="https://www.echobox.com" rel="nofollow">Echobox</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'u

In [18]:
tweet_1.find_one({"$text": {"$search": "College"}})

{'_id': ObjectId('63bcec33b86a9ea17db72f9b'),
 'created_at': 'Sun Jan 08 07:33:10 +0000 2023',
 'id': 1611989373737906183,
 'id_str': '1611989373737906183',
 'text': 'Talking to media at SMS Medical College | January 08. https://t.co/sjiIq7dkQu',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/sjiIq7dkQu',
    'expanded_url': 'https://twitter.com/i/broadcasts/1ZkKzXpnnpWJv',
    'display_url': 'twitter.com/i/broadcasts/1…',
    'indices': [54, 77]}]},
 'metadata': {'result_type': 'popular', 'iso_language_code': 'en'},
 'source': '<a href="https://studio.twitter.com" rel="nofollow">Twitter Media Studio</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 385926583,
  'id_str': '385926583',
  'name': 'Ashok Gehlot',
  'screen_name': 'ashokgehlot51',
  'location': 'Rajasthan, India

In [19]:
tweet_1.find_one({"$text": {"$search": "College"}})["text"]

'Talking to media at SMS Medical College | January 08. https://t.co/sjiIq7dkQu'

In [20]:
tweet_1.find_one({"$text": {"$search": "College"}})["user"]["name"]

'Ashok Gehlot'

In [21]:
tweet_1.find_one({"$text": {"$search": "College"}})["user"]["description"]

'Chief Minister of Rajasthan'

In [22]:
tweet_1.find_one({"$text": {"$search": "College"}})["user"]["followers_count"]

4436337

In [23]:
tweet_1.find_one({"$text": {"$search": "College"}})["user"]["friends_count"]

106

In [24]:
tweet_1.find_one({"$text": {"$search": "College"}})["user"]["statuses_count"]

33584

In [25]:
tweet_1.find_one({"$text": {"$search": "College"}})["user"]["created_at"]

'Thu Oct 06 11:27:58 +0000 2011'

In [26]:
tweet_1.find_one({"$text": {"$search": "College"}})["user"]["favourites_count"]

11151

In [27]:
tweet_1.find_one({"$text": {"$search": "College"}})["user"]["verified"]

True

In [28]:
tweet_1.find_one({"$text": {"$search": "College"}})["user"]["geo_enabled"]

False

In [29]:
tweet_1.find_one({"$text": {"$search": "College"}})["user"]["profile_image_url"]

'http://pbs.twimg.com/profile_images/1562365094830682114/Hk6nxBPq_normal.jpg'